# Assignment 3  - AI tools 

## Part 2 - Implementation 

In [2]:
import pandas as pd
import numpy as np

In [3]:
# merge data into one dataframe and add names 
beijing = pd.read_csv('Beijing_labeled.csv')
beijing['City']='beijing'

chengdu = pd.read_csv('Chengdu_labeled.csv')
chengdu['City']='chengdu'

guangzhou = pd.read_csv('Guangzhou_labeled.csv')
guangzhou['City']='guangzhou'

shanghai = pd.read_csv('Shanghai_labeled.csv')
shanghai['City']='shanghai'

shenyang = pd.read_csv('Shenyang_labeled.csv')
shenyang['City']='shenyang'

all_data = pd.concat([beijing,chengdu,guangzhou,shanghai, shenyang])

print(all_data)

     season  DEWP   HUMI    PRES  TEMP     Iws  precipitation  cbwd_NE  \
0       4.0  -8.0  79.00  1026.0  -5.0   23.69            0.0        0   
1       4.0 -11.0  85.00  1021.0  -9.0  105.93            1.1        0   
2       4.0 -21.0  43.00  1030.0 -11.0  117.55            0.0        0   
3       4.0 -25.0  33.00  1034.0 -12.0   39.35            0.0        1   
4       4.0 -24.0  30.00  1034.0 -10.0   59.00            0.0        1   
..      ...   ...    ...     ...   ...     ...            ...      ...   
819     4.0 -22.0  50.63  1038.0 -14.0    4.00            0.0        0   
820     4.0 -16.0  52.47  1034.0  -8.0    4.00            0.0        0   
821     4.0 -13.0  45.91  1028.0  -3.0   40.00            0.0        0   
822     4.0  -4.0  80.04  1029.0  -1.0    6.00            0.0        0   
823     4.0  -8.0  79.42  1033.0  -5.0    4.00            0.0        0   

     cbwd_NW  cbwd_SE  PM_HIGH      City  
0          0        1      1.0   beijing  
1          0        1    

## Karls version

def get_city_data(city,data = all_data):
    return all_data[all_data['City'].isin(city)].drop('City', axis=1).astype(float)

def standardize_data(data):
    data_copy = data.copy()
    for col in (list(data_copy.columns.values)):
        data_copy[col]=(data_copy[col] - data_copy[col].mean() ) / data_copy[col].std()
    return data_copy

def normalize_data(data):
    data_copy = data.copy()
    for col in (list(data_copy.columns.values)):
        data_copy[col] = (data_copy[col] - data_copy[col].min()) / (data_copy[col].max() - data_copy[col].min())    
    return data_copy

def split_data(data):
    X = data.drop('PM_HIGH', axis=1)
    Y = data['PM_HIGH']
    return X,Y

class KNN():
    def __init__(self, k = 5, dist = 'euclidian'):
        self.k = k
        self.X_train = None
        self.Y_train = None
        self.dist = dist

    def fit(self, x, y):
        self.X_train = x
        self.Y_train = y

    def distance(self,x1,x2):
        if self.dist == 'euclidian':
            return np.sqrt(np.sum((x1-x2)**2))

        elif self.dist == 'manhattan':
            return sum(abs(val1-val2) for val1, val2 in zip(x1,x2))

        else:
            print('Error: faulty distance measure')

    def predict(self, X_test):
        labels = np.zeros(X_test.shape[0]) 
        print(self.k)
        for i, x in X_test.reset_index().iterrows():  
            distances = self.distance(self.X_train, x) 
            neighbors_labels = self.Y_train[np.argsort(distances)[:self.k]] 
            unique, counts = np.unique(neighbors_labels, return_counts=True) 
            labels[i] = unique[np.argmax(counts)]
        return labels

    def score(self, X_test, Y_test):
        y_pred = self.predict(X_test)
        accuracy = sum(y_pred == Y_test) / len(Y_test)
        return accuracy
        

## Kais version

General tips from the lecture: Don't write one long script, use modular blocks that can be reused. + Use: fit, predict, score pattern

- https://machinelearningmastery.com/tutorial-to-implement-k-nearest-neighbors-in-python-from-scratch/

In [39]:
from math import sqrt
#use euclidian distance? The table should allow it

#HERE WE CAN ADD MORE DISTANCE MEASURES
# calculate the Euclidean distance between two vectors
def euclidean_distance(row1, row2):
    distance = 0.0
    for i in range(len(row1)): #obs! without lable now (len(row1)-1) before
        distance += (row1[i] - row2[i])**2
    return sqrt(distance)

In [40]:
def get_neighbors(train, test_rows, num_neighbors=25):
    distances = list()
    neighborlist = list()
    #num_neighbors = 25 # default value
    
    for test_row in test_rows:

        for train_row in train:
            dist = euclidean_distance(test_rows, train_row)
            distances.append((train_row, dist))
            distances.sort(key=lambda tup: tup[1]) #custom key is used ensuring that the second item in the tuple (tup[1]) is used in the sorting operation.
    
        neighbors = list()
        
        for i in range(num_neighbors):
            neighbors.append(distances[i][0])

        for i in range(test_rows):      
            neighborlist.append(neighbors[i][0])

    return neighborlist

In [41]:
# Calculate accuracy percentage
def accuracy_metric(actual, predicted):
	correct = 0
	for i in range(len(actual)):
		if actual[i] == predicted[i]:
			correct += 1
	return correct / float(len(actual)) * 100.0

In [42]:
# how is the inheritance declared? && arguments needed for default vals?
class k_means_clas:
    
    def __init__(self,x_train, y_train): # superfluous
        self.x_train=x_train 
        self.y_train=y_train
    
    #traing and storing model parameters that optimize the objective --> OLS or K-NN
    #KNN only stores training data
    def fit(self, training_data):
        neighborhood = []
        neighborhood = training_data #model only stores the whole training data
        
    #should take a (new) data point and predict the outcome
    def predict(self, x_train, y_train, new_points, num_neighbors): # new points is list
        x_train['label'] = y_train 
        train_data = x_train
        neighbors = get_neighbors(train=train_data, test_rows=new_points, num_neighbors= num_neighbors) # giving it train data again? 
        output_values = [row[-1] for row in neighbors]
        prediction = max(set(output_values), key=output_values.count)
        return prediction
    
    #assigns a score to the prediction made for x in comparison to the label y
    def score(self, x_test,y_test): 
        scores = accuracy_metric(predicted=x_test, actual= y_test)
        #scores = evaluate_algorithm(dataset, k_nearest_neighbors, n_folds, num_neighbors) # less arguments?
        return scores
    

In [56]:
city_data = get_city_data(['shanghai'])
standardized = standardize_data(city_data)
X_train, X_test, Y_train, Y_test = split_data(standardized)
# instanciate the object
C = k_means_clas(X_train, Y_train)
#C.fit(X_train, Y_train) # fit should normally do something... But we don't seem to need it for KNN
#Unlike other Machine Learning Algorithms KNN doesn't optimize a cost function instead it remembers the training data. When a prediction is made the KNN compares the input with the training data it has stored. The class label of the data point which has maximum similarity with the queried input is given as prediction. Hence when we fit a KNN model it learns or stores the dataset in memory.

C.predict(X_train, Y_train, new_points=X_test, num_neighbors=10)
C.score(X_test,Y_test)
  

KeyError: 0

In [ ]:
#nästa steg när koden ovan funkar

train_data = get_city_data(['beijing','shenyang'])
test_data = get_city_data(['guangzhou','shanghai'])

strd_train = standardize_data(train_data)
strd_test = standardize_data(test_data)

norm_train = normalize_data(train_data)
norm_test = normalize_data(test_data)

X_train, Y_train = split_data(strd_train)
X_test, Y_test = split_data(strd_train)



for data in [city_data,standardized,normalized]:
    X_train, X_test, Y_train, Y_test = split_data(data)
    clf = KNN(k=10)
    clf.fit(X_train,Y_train)
    acc = clf.score(X_test, Y_test)
    print(acc)


NameError: name 'Classifier' is not defined

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=9c52a5d0-b452-40c9-bc0a-977a02670dfa' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>